# Código Python para buscar publicaçoes Youtube e Gerar os arquivos .csv no formato normalizado e com label 

In [ ]:
# Importando as bibliotecas necessárias
import os
import pandas as pd
from googleapiclient.discovery import build


In [ ]:
# Função para buscar vídeos com informações detalhadas dentro de um intervalo de datas
def search_youtube_videos(api_key, query, published_after, published_before):
    youtube = build('youtube', 'v3', developerKey=api_key)
    videos = []
    next_page_token = None
    
    while True:
        try:
            response = youtube.search().list(
                q=query,
                part='snippet',
                type='video',
                regionCode='BR',
                publishedAfter=published_after,
                publishedBefore=published_before,
                maxResults=50,  # Máximo de resultados por página
                pageToken=next_page_token
            ).execute()
        except Exception as e:
            print(f"Erro ao buscar vídeos: {e}")
            raise
        
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            
            # Obter detalhes adicionais do vídeo
            try:
                video_response = youtube.videos().list(
                    part='snippet,statistics,contentDetails',
                    id=video_id
                ).execute()
                
                video_info = video_response['items'][0]
                video_data = {
                    'video_id': video_id,
                    'url': video_url,
                    'title': video_info['snippet']['title'],
                    'description': video_info['snippet']['description'],
                    'channel_title': video_info['snippet']['channelTitle'],
                    'published_at': video_info['snippet']['publishedAt'],
                    'view_count': video_info['statistics'].get('viewCount', 0),
                    'like_count': video_info['statistics'].get('likeCount', 0),
                    'dislike_count': video_info['statistics'].get('dislikeCount', 0),
                    'comment_count': video_info['statistics'].get('commentCount', 0),
                    'tags': video_info['snippet'].get('tags', []),
                    'duration': video_info['contentDetails']['duration'],
                    'sentiment': None  # Nenhuma informação disponível na API sobre sentimento
                }
                videos.append(video_data)
            except Exception as e:
                print(f"Erro ao obter detalhes do vídeo {video_id}: {e}")
                continue
        
        next_page_token = response.get('nextPageToken')
        
        if not next_page_token:
            break
    
    return videos

In [ ]:
# Função para organizar os dados dos vídeos conforme especificado
def organize_video_data(videos):
    organized_data = []
    for video in videos:
        # Convertendo o valor numérico de SENTIMENT para a string correspondente
        sentiment_mapping = {0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}
        sentiment = sentiment_mapping.get(video.get('sentiment'), None)
        
        organized_video = {
            'ID': video['video_id'],
            'AUTHOR': video['published_at'],
            'URL': video['url'],
            'DATE': video['published_at'],
            'LIKES': video['like_count'],
            'SHARES': None,  # Nenhuma informação disponível na API sobre compartilhamentos
            'COMMENTS': video['comment_count'],
            'CONTENT': video['description'],
            'SERVICE': 'Youtube',
            'SENTIMENT': sentiment,
            'ENGAGEMENT': None  # Nenhuma informação disponível na API sobre engajamento
        }
        organized_data.append(organized_video)
    return organized_data

In [ ]:
# Função para converter o sentimento para número
def convert_sentiment(sentiment):
    if sentiment == 'NEUTRAL':
        return 0
    elif sentiment == 'POSITIVE':
        return 1
    elif sentiment == 'NEGATIVE':
        return 2
    else:
        return None

In [ ]:
# Criar um diretório para os arquivos CSV, se não existir
output_directory = 'output_csv'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
# Exemplo de uso
api_key = 'AIzaSyB5YpHY8DKSrEHLvflA88XgE0wsi5uF5f0PI_KEY'
query = 'Show Madonna Copacapana'
published_after = '2024-01-01T00:00:00Z'  # Data de início do intervalo de busca (formato ISO 8601)
published_before = '2024-06-13T23:59:59Z'  # Data de término do intervalo de busca (formato ISO 8601)

In [ ]:
# Buscar vídeos
videos = search_youtube_videos(api_key, query, published_after, published_before)

In [ ]:
# Organizar os dados
organized_videos = organize_video_data(videos)

In [ ]:
# Converter os dados organizados em arquivos CSV individuais
for index, video in enumerate(organized_videos):
    # Obter os dados do vídeo
    video_id = video['ID']
    service = video['SERVICE']
    date = pd.to_datetime(video['DATE']).strftime('%Y%m%d%H%M')
    sentiment = convert_sentiment(video['SENTIMENT'])
    index_formatted = str(index).zfill(7)  # Formato do índice: 0000000

    # Construir o nome do arquivo
    filename = f"{service}_{date}_{sentiment}_{index_formatted}.csv"
    filepath = os.path.join(output_directory, filename)

    # Criar DataFrame com apenas uma linha
    df_single_row = pd.DataFrame([video])

    # Salvar o DataFrame como arquivo CSV
    df_single_row.to_csv(filepath, index=False)

# Update de codigo para ajustar entendimento da ultima reunião de 20240613. #